In [8]:
#creditials to begin connection
import pandas as pd
import mysql.connector
from mysql.connector import Error


host = '127.0.0.1'
user = 'Felipe'
password = 'JUULisCUUL420!'  
database = 'SchoolDB'


csv_file_path = '/Users/felipequiroz/Desktop/DS5110/DataProject/responses.csv'


In [10]:
#begin connection
try:
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    if connection.is_connected():
        print("Connected to MySQL database")
        cursor = connection.cursor()

        df = pd.read_csv(csv_file_path)
        print("CSV file loaded successfully")

        
        df = df.where(pd.notnull(df), None)

        #begin loading in data per table
        #Load into students
        for _, row in df.iterrows():
            cursor.execute("""
                INSERT INTO Students (student_id, name, email)
                VALUES (%s, %s, %s)
                ON DUPLICATE KEY UPDATE name=VALUES(name), email=VALUES(email)
            """, (row['student_id'], row['name'], row['email']))

        #Load into Classes
        for _, row in df.iterrows():
            cursor.execute("""
                INSERT INTO Classes (class_id, class_name, credits, department)
                VALUES (%s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE class_name=VALUES(class_name), credits=VALUES(credits), department=VALUES(department)
            """, (row['class_id'], row['class_name'], row['credits'], row['department']))

        #Load into Student_classes
        for _, row in df.iterrows():
            cursor.execute("""
                INSERT INTO Student_Classes (student_id, class_id, can_tutor, tutoring_availability)
                VALUES (%s, %s, %s, %s)
            """, (
                row['student_id'], row['class_id'],
                row['can_tutor'] == "Yes",  
                row['tutoring_availability']
            ))

        #Load into Course_notes
        for _, row in df.iterrows():
            if not pd.isnull(row['note_id']):  
                cursor.execute("""
                    INSERT INTO Course_Notes (note_id, class_id, note_title, content)
                    VALUES (%s, %s, %s, %s)
                """, (row['note_id'], row['class_id'], row['note_title'], row['content']))

        #Load into Notebooks        
        for _, row in df.iterrows():
            if not pd.isnull(row['notebook_id']):  
                cursor.execute("""
                    INSERT INTO Notebooks (notebook_id, class_id, title, file)
                    VALUES (%s, %s, %s, %s)
                """, (row['notebook_id'], row['class_id'], row['title'], row['file']))

        connection.commit()
        print("All data inserted successfully into the database")

except Error as e:
    print(f"Error: {e}")

finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed")


Connected to MySQL database
CSV file loaded successfully
Error: 1062 (23000): Duplicate entry '1' for key 'course_notes.PRIMARY'
MySQL connection closed


In [6]:
#demonstrate that data has been loaded in correctly per table
try:
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    if connection.is_connected():
        print("Connected to MySQL database")
        cursor = connection.cursor()
        
        #show students table
        cursor.execute("SELECT * FROM Students;")
        students = cursor.fetchall()
        print("Students Table:")
        for row in students:
            print(row)

        #show classes table
        cursor.execute("SELECT * FROM Classes;")
        classes = cursor.fetchall()
        print("\nClasses Table:")
        for row in classes:
            print(row)

        #show student_classes table
        cursor.execute("SELECT * FROM Student_Classes;")
        student_classes = cursor.fetchall()
        print("\nStudent_Classes Table:")
        for row in student_classes:
            print(row)

        #show course_notes table
        cursor.execute("SELECT * FROM Course_Notes;")
        course_notes = cursor.fetchall()
        print("\nCourse_Notes Table:")
        for row in course_notes:
            print(row)

        #show notebooks table
        cursor.execute("SELECT * FROM Notebooks;")
        notebooks = cursor.fetchall()
        print("\nNotebooks Table:")
        for row in notebooks:
            print(row)

except Error as e:
    print(f"Error: {e}")

finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed")

Connected to MySQL database
Students Table:
(1, 'Felipe Quiroz', 'quiroz.w@northeastern.edu')
(2, 'Kevin Cooper', 'cooper.ke@northeastern.edu')
(3, 'Jon Jones', 'bones@ufc.com')
(4, 'Tony Ferguson', 'tony@ufc.com')

Classes Table:
('ds5110', 'Intro to Data Management and Processing', 'Khoury', 4)

Student_Classes Table:
(1, 1, 'ds5110', '1', 'I can tutor only on Friday mornings')
(2, 2, 'ds5110', '1', 'Email me for tutoring')
(3, 3, 'ds5110', '0', None)
(4, 4, 'ds5110', '0', None)

Course_Notes Table:
(1, 'ds5110', 'syllabus', '/Users/felipequiroz/Desktop/DS5110/DataProject/DS 5110_Syllabus_Fall2024-3.pdf')
(2, 'ds5110', 'commands video', '/Users/felipequiroz/Desktop/DS5110/DataProject/IMG_6859.MOV')

Notebooks Table:
(1, 'ds5110', 'erm description', '/Users/felipequiroz/Desktop/DS5110/DataProject/Screenshot 2024-11-25 at 5.13.25\u202fAM.png')
(2, 'ds5110', 'schedule', '/Users/felipequiroz/Desktop/DS5110/DataProject/Fall_2024Schedule-FatemaNafa.pdf')
MySQL connection closed
